In [ ]:
# from planetsca import data_gathering as dg
# from planetsca import data_preparation as dp
# from planetsca import model_training as mt
# from planetsca import prediction_evaluation as pe

import glob

import matplotlib.pyplot as plt
import xarray as xr

import planetsca as ps

In [ ]:
df_train = ps.data_preparation.data_training_existing("sample_100k.csv")

In [ ]:
df_train.head()

In [ ]:
model_filepath = "random_forest_20240116_binary_174K.joblib"
model_score_filepath = "random_forest_20240116_binary_174K_scores.csv"

In [ ]:
ps.model_training.train_model(
    model_filepath,
    model_score_filepath,
    n_estimators=10,
    max_depth=10,
    max_features=4,
    random_state=1,
    n_splits=2,
    n_repeats=1,
    df_train=df_train,
)

In [ ]:
ps_image_filepaths = glob.glob("demo_images*/*/PSScene/*SR_clip.tif")

In [ ]:
output_dirpath = "./demo_images/SCA/"

In [ ]:
sca_image_paths = ps.predict.predict_sca(
    planet_path=ps_image_filepaths,
    model_filepath=model_filepath,
    output_dirpath=output_dirpath,
)

In [ ]:
ps_image_filepaths.sort()
sca_image_paths.sort()

for ps_image_filepath, sca_image_filepath in zip(ps_image_filepaths, sca_image_paths):
    ps_image = xr.open_dataset(ps_image_filepath)
    sca_image = xr.open_dataset(sca_image_filepath)

    fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), tight_layout=True)
    ps_image.isel(band=0).band_data.plot(ax=ax1, cmap="Greys_r", add_colorbar=False)
    sca_image.isel(band=0).band_data.plot(ax=ax2, cmap="Greys_r", add_colorbar=False)

    ax1.set_title(ps_image_filepath.split("/")[-1])
    ax2.set_title(sca_image_filepath.split("/")[-1])